In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
# -------- TEST USER ----------- #

TEST_USER      = '007'

BASE_DIR       = '../'
IMG_DIR        = 'Spectrogram-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']
AXES           = ['X', 'Y', 'Z']

BATCH_SIZE     = 32
IMG_LEN        = 160
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 15
LEARNING_RATE  = 0.001

In [3]:
def load_data(axis):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for the ' + gesture + ' gesture on the '  + axis + ' axis ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, axis, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

In [4]:
X_train_x, X_test_x, y_train_x, y_test_x = load_data('X')
X_train_y, X_test_y, y_train_y, y_test_y = load_data('Y')
X_train_z, X_test_z, y_train_z, y_test_z = load_data('Z')

loading data for the j gesture on the X axis ... √
loading data for the z gesture on the X axis ... √
loading data for the bad gesture on the X axis ... √
loading data for the deaf gesture on the X axis ... √
loading data for the fine gesture on the X axis ... √
loading data for the good gesture on the X axis ... √
loading data for the goodbye gesture on the X axis ... √
loading data for the hello gesture on the X axis ... √
loading data for the hungry gesture on the X axis ... √
loading data for the me gesture on the X axis ... √
loading data for the no gesture on the X axis ... √
loading data for the please gesture on the X axis ... √
loading data for the sorry gesture on the X axis ... √
loading data for the thankyou gesture on the X axis ... √
loading data for the yes gesture on the X axis ... √
loading data for the you gesture on the X axis ... √
loading data for the j gesture on the Y axis ... √
loading data for the z gesture on the Y axis ... √
loading data for the bad gesture o

In [5]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [6]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(GESTURES))

In [8]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [9]:
model_x = get_model()
history_x = model_x.fit(X_train_x, y_train_x, epochs=EPOCHS)

Epoch 1/15
30/30 [==============================] - 12s 339ms/step - loss: 3.1128 - accuracy: 0.0671
Epoch 2/15
30/30 [==============================] - 10s 333ms/step - loss: 2.5690 - accuracy: 0.1965
Epoch 3/15
30/30 [==============================] - 10s 334ms/step - loss: 2.2808 - accuracy: 0.2584
Epoch 4/15
30/30 [==============================] - 10s 332ms/step - loss: 2.0895 - accuracy: 0.3320
Epoch 5/15
30/30 [==============================] - 10s 332ms/step - loss: 1.9454 - accuracy: 0.3508
Epoch 6/15
30/30 [==============================] - 10s 334ms/step - loss: 1.8403 - accuracy: 0.3954
Epoch 7/15
30/30 [==============================] - 10s 337ms/step - loss: 1.7582 - accuracy: 0.4458
Epoch 8/15
30/30 [==============================] - 10s 339ms/step - loss: 1.6743 - accuracy: 0.4616
Epoch 9/15
30/30 [==============================] - 10s 340ms/step - loss: 1.6334 - accuracy: 0.4873
Epoch 10/15
30/30 [==============================] - 10s 340ms/step - loss: 1.4946 - accura

In [10]:
prob_x = tf.keras.Sequential([model_x, tf.keras.layers.Softmax()])
y_pred_x = prob_x.predict(X_test_x)
y_pred = np.argmax(y_pred_x, axis=1)
print(classification_report(y_test_x.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.67      0.20      0.31        10
         1.0       0.38      0.30      0.33        10
         2.0       0.00      0.00      0.00        10
         3.0       0.00      0.00      0.00        10
         4.0       0.33      0.30      0.32        10
         5.0       0.00      0.00      0.00        10
         6.0       0.43      1.00      0.61        10
         7.0       0.00      0.00      0.00        10
         8.0       0.36      0.40      0.38        10
         9.0       0.56      0.50      0.53        10
        10.0       0.30      0.30      0.30        10
        11.0       0.60      0.60      0.60        10
        12.0       0.50      0.40      0.44        10
        13.0       0.29      0.40      0.33        10
        14.0       0.43      0.60      0.50        10
        15.0       0.40      0.20      0.27        10

    accuracy                           0.33       160
   macro avg       0.33   

In [11]:
model_y = get_model()
history_y = model_y.fit(X_train_y, y_train_y, epochs=EPOCHS)

Epoch 1/15
30/30 [==============================] - 12s 332ms/step - loss: 2.3915 - accuracy: 0.2480
Epoch 2/15
30/30 [==============================] - 10s 330ms/step - loss: 2.0594 - accuracy: 0.3131
Epoch 3/15
30/30 [==============================] - 10s 333ms/step - loss: 1.8577 - accuracy: 0.3755
Epoch 4/15
30/30 [==============================] - 10s 337ms/step - loss: 1.7147 - accuracy: 0.4193
Epoch 5/15
30/30 [==============================] - 10s 330ms/step - loss: 1.6133 - accuracy: 0.4459
Epoch 6/15
30/30 [==============================] - 10s 334ms/step - loss: 1.5197 - accuracy: 0.4939
Epoch 7/15
30/30 [==============================] - 10s 330ms/step - loss: 1.4627 - accuracy: 0.5179
Epoch 8/15
30/30 [==============================] - 10s 334ms/step - loss: 1.3510 - accuracy: 0.5498
Epoch 9/15
30/30 [==============================] - 10s 328ms/step - loss: 1.3699 - accuracy: 0.5352
Epoch 10/15
30/30 [==============================] - 10s 334ms/step - loss: 1.2750 - accura

In [12]:
prob_y = tf.keras.Sequential([model_y, tf.keras.layers.Softmax()])
y_pred_y = prob_y.predict(X_test_y)
y_pred = np.argmax(y_pred_y, axis=1)
print(classification_report(y_test_y.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.36      0.40      0.38        10
         1.0       0.54      0.70      0.61        10
         2.0       0.36      0.40      0.38        10
         3.0       0.23      0.30      0.26        10
         4.0       0.33      0.10      0.15        10
         5.0       0.17      0.10      0.12        10
         6.0       0.60      0.90      0.72        10
         7.0       0.00      0.00      0.00        10
         8.0       0.86      0.60      0.71        10
         9.0       0.33      0.10      0.15        10
        10.0       0.78      0.70      0.74        10
        11.0       0.14      0.20      0.17        10
        12.0       0.28      0.50      0.36        10
        13.0       0.50      0.20      0.29        10
        14.0       0.62      0.50      0.56        10
        15.0       0.27      0.60      0.37        10

    accuracy                           0.39       160
   macro avg       0.40   

In [13]:
model_z = get_model()
history_z = model_z.fit(X_train_z, y_train_z, epochs=EPOCHS)

Epoch 1/15
30/30 [==============================] - 12s 335ms/step - loss: 2.6527 - accuracy: 0.2435
Epoch 2/15
30/30 [==============================] - 10s 334ms/step - loss: 2.3114 - accuracy: 0.2650
Epoch 3/15
30/30 [==============================] - 10s 334ms/step - loss: 2.0002 - accuracy: 0.3605
Epoch 4/15
30/30 [==============================] - 10s 339ms/step - loss: 1.8013 - accuracy: 0.4114
Epoch 5/15
30/30 [==============================] - 10s 337ms/step - loss: 1.7304 - accuracy: 0.4395
Epoch 6/15
30/30 [==============================] - 10s 339ms/step - loss: 1.6073 - accuracy: 0.4745
Epoch 7/15
30/30 [==============================] - 10s 341ms/step - loss: 1.4563 - accuracy: 0.5178
Epoch 8/15
30/30 [==============================] - 10s 344ms/step - loss: 1.4144 - accuracy: 0.5295
Epoch 9/15
30/30 [==============================] - 10s 343ms/step - loss: 1.3586 - accuracy: 0.5366
Epoch 10/15
30/30 [==============================] - 10s 344ms/step - loss: 1.3134 - accura

In [14]:
prob_z = tf.keras.Sequential([model_z, tf.keras.layers.Softmax()])
y_pred_z = prob_z.predict(X_test_z)
y_pred = np.argmax(y_pred_z, axis=1)
print(classification_report(y_test_z.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.33      0.10      0.15        10
         1.0       0.40      0.60      0.48        10
         2.0       0.19      0.30      0.23        10
         3.0       0.45      0.50      0.48        10
         4.0       0.25      0.10      0.14        10
         5.0       0.25      0.10      0.14        10
         6.0       0.59      1.00      0.74        10
         7.0       0.23      0.30      0.26        10
         8.0       0.88      0.70      0.78        10
         9.0       0.50      0.40      0.44        10
        10.0       0.60      0.30      0.40        10
        11.0       0.00      0.00      0.00        10
        12.0       0.50      0.50      0.50        10
        13.0       0.44      0.40      0.42        10
        14.0       0.59      1.00      0.74        10
        15.0       0.64      0.70      0.67        10

    accuracy                           0.44       160
   macro avg       0.43   

In [15]:
y_total = y_pred_x * y_pred_y * y_pred_z
y_pred = np.argmax(y_total, axis=1)
report = classification_report(y_test_x.ravel(), y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0       0.67      0.20      0.31        10
         1.0       0.62      0.80      0.70        10
         2.0       0.33      0.40      0.36        10
         3.0       0.50      0.60      0.55        10
         4.0       1.00      0.70      0.82        10
         5.0       0.25      0.10      0.14        10
         6.0       0.62      1.00      0.77        10
         7.0       0.12      0.10      0.11        10
         8.0       0.75      0.90      0.82        10
         9.0       1.00      0.80      0.89        10
        10.0       0.80      0.40      0.53        10
        11.0       0.17      0.10      0.12        10
        12.0       0.43      0.60      0.50        10
        13.0       0.57      0.80      0.67        10
        14.0       0.56      1.00      0.71        10
        15.0       0.75      0.60      0.67        10

    accuracy                           0.57       160
   macro avg       0.57   

In [16]:
config = '\n\nTEST_USER ' + TEST_USER + '\n'
underline = '=====================================\n'
log_dir = os.path.join(BASE_DIR, LOG_DIR)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
f = open(os.path.join(log_dir, 'spg_logs.txt'), 'a')
f.write(config)
f.write(underline)
f.write(report)
f.close()

In [ ]:
'''
         
         SPECTROGRAM TRANSFER LEARNING
        
            TEST USER       ACCURACY
        ---------------------------------
               001             54%
               002             61%
               003             63%
               004             41%
               005             48%
               006             61%
               007             57%

'''